In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("prior_train_orders.csv")
df = df[df['eval_set'] == 'train']
df

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,department,aisle
8,3817,38784.0,1.0,1.0,Gluten Free Pizza Crust Whole Grain Mix,105.0,13.0,7172.0,train,23.0,3.0,13.0,7.0,pantry,doughs gelatins bake mixes
14,6096,32176.0,5.0,1.0,10 Grain Bread Mix,105.0,13.0,5965.0,train,6.0,2.0,18.0,9.0,pantry,doughs gelatins bake mixes
28,12983,9752.0,8.0,0.0,Lime Gelatin,105.0,13.0,3811.0,train,6.0,3.0,19.0,30.0,pantry,doughs gelatins bake mixes
42,20018,38784.0,5.0,1.0,Gluten Free Pizza Crust Whole Grain Mix,105.0,13.0,2309.0,train,10.0,2.0,12.0,16.0,pantry,doughs gelatins bake mixes
48,22290,23264.0,5.0,0.0,Original Corn Bread Mix,105.0,13.0,7086.0,train,14.0,0.0,10.0,30.0,pantry,doughs gelatins bake mixes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79628,592557,13176.0,13.0,1.0,Bag of Organic Bananas,24.0,4.0,866.0,train,23.0,0.0,16.0,15.0,produce,fresh fruits
79672,597879,3798.0,7.0,0.0,Pink Lady Apples,24.0,4.0,9564.0,train,4.0,2.0,19.0,30.0,produce,fresh fruits
79673,598000,13176.0,9.0,1.0,Bag of Organic Bananas,24.0,4.0,7207.0,train,34.0,1.0,9.0,8.0,produce,fresh fruits
79685,599318,5876.0,8.0,1.0,Organic Lemon,24.0,4.0,9868.0,train,22.0,0.0,10.0,21.0,produce,fresh fruits


### Create a basket
As the dataset contains huge amout of data, let us take a subset of the data to extract the association rules from it.

**Assumptions**: Segment the data by considering the 100 most frequent ordered items. Please note it is just an assumption. You can consider 'n frequent order items as per your choice.

In [ ]:
product_counts = df.groupby('product_id')['order_id'].count().reset_index().rename(columns = {'order_id':'frequency'})
product_counts = product_counts.sort_values('frequency', ascending=False)[0:100].reset_index(drop=True)
product_counts


,product_id,frequency
0,13176.0,132
1,21136.0,99
2,21904.0,95
3,16800.0,56
4,45056.0,41
...,...,...
95,10960.0,7
96,45664.0,7
97,18992.0,7
98,39520.0,7


In [ ]:
freq_products = list(product_counts.product_id)
del product_counts 
freq_products[1:10]


[21136.0,
 21904.0,
 16800.0,
 45056.0,
 5876.0,
 37632.0,
 26208.0,
 42272.0,
 43968.0]

In [ ]:
order_products = df[df.product_id.isin(freq_products)]
del df
order_products.shape


(1687, 15)

In [ ]:
df=order_products[['order_id','product_name','reordered']].set_index('order_id')
df

,product_name,reordered
order_id,,
3817,Gluten Free Pizza Crust Whole Grain Mix,1.0
6096,10 Grain Bread Mix,1.0
20018,Gluten Free Pizza Crust Whole Grain Mix,1.0
30739,Gluten Free Pizza Crust Whole Grain Mix,1.0
36665,10 Grain Bread Mix,0.0
...,...,...
586038,Bag of Organic Bananas,1.0
592557,Bag of Organic Bananas,1.0
598000,Bag of Organic Bananas,1.0


In [ ]:
basket = df.pivot_table(columns='product_name', values='reordered', index='order_id').reset_index().fillna(0).set_index('order_id')
basket

product_name,10 Grain Bread Mix,100% Whey Protein Powder Vanilla,All Natural Bergamot Lime Vegan Deodorant,Baby Back Pork Ribs,Bag of Organic Bananas,Bandaid Securflex Wrap,Beef Fajitas,Body Deodorant Stick Fragrance & Paraben Free,Bourbon Brown Sugar BBQ Sauce,Caesar Croutons,...,Ultra Apple Blossom Anti-Bacterial Dish Soap,Vegan Lemon Herb Salad Dressing,Vitacraves Multivitamin Supplement Gummies for Adults,Vitamin C Refreshing Facial Cleanser,Vitamin C Rejuvenating Oil-Free Moisturizer,Vitamin Code Men's Raw Whole Food Mutlivitamin,Vitamin D3 Gummies,White Beauty Bar Soap,Wild Collection Wolfthorn Body Spray,Wild Rose Smoothing Eye Cream
order_id,,,,,,,,,,,,,,,,,,,,,
988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1890,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599726,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
599773,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
600173,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
    
basket = basket.applymap(encode_units)
basket.head()




product_name,10 Grain Bread Mix,100% Whey Protein Powder Vanilla,All Natural Bergamot Lime Vegan Deodorant,Baby Back Pork Ribs,Bag of Organic Bananas,Bandaid Securflex Wrap,Beef Fajitas,Body Deodorant Stick Fragrance & Paraben Free,Bourbon Brown Sugar BBQ Sauce,Caesar Croutons,...,Ultra Apple Blossom Anti-Bacterial Dish Soap,Vegan Lemon Herb Salad Dressing,Vitacraves Multivitamin Supplement Gummies for Adults,Vitamin C Refreshing Facial Cleanser,Vitamin C Rejuvenating Oil-Free Moisturizer,Vitamin Code Men's Raw Whole Food Mutlivitamin,Vitamin D3 Gummies,White Beauty Bar Soap,Wild Collection Wolfthorn Body Spray,Wild Rose Smoothing Eye Cream
order_id,,,,,,,,,,,,,,,,,,,,,
988,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1275,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1865,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1890,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1994,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Apply Apriori algorithm
As the dataset contains huge amount of data, let us take a subset of the data to extract the association rules from it.

**Assumptions**: Segment the basket by considering 100000 record. Please note its just an assumption, you can consider 'n' records as per choice.

In [ ]:
shortbasket = basket[:100000]
shortbasket

product_name,10 Grain Bread Mix,100% Whey Protein Powder Vanilla,All Natural Bergamot Lime Vegan Deodorant,Baby Back Pork Ribs,Bag of Organic Bananas,Bandaid Securflex Wrap,Beef Fajitas,Body Deodorant Stick Fragrance & Paraben Free,Bourbon Brown Sugar BBQ Sauce,Caesar Croutons,...,Ultra Apple Blossom Anti-Bacterial Dish Soap,Vegan Lemon Herb Salad Dressing,Vitacraves Multivitamin Supplement Gummies for Adults,Vitamin C Refreshing Facial Cleanser,Vitamin C Rejuvenating Oil-Free Moisturizer,Vitamin Code Men's Raw Whole Food Mutlivitamin,Vitamin D3 Gummies,White Beauty Bar Soap,Wild Collection Wolfthorn Body Spray,Wild Rose Smoothing Eye Cream
order_id,,,,,,,,,,,,,,,,,,,,,
988,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1275,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1865,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1890,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1994,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599726,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
599773,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
600173,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


####  Apriori is a popular algorithm  for extracting frequent itemsets with applications in association rule learning. The apriori algorithm has been designed to operate on databases containing transactions, such as purchases by customers of a store. An itemset is considered as "frequent" if it meets a user-specified support threshold.

For instance, if the support threshold is set to 0.5 (50%), a frequent itemset is defined as a set of items that occur together in at least 50% of all transactions in the database.

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_items = apriori(shortbasket, min_support=0.001, use_colnames=True , verbose =1 , low_memory=True)

# The length column has been added to increase ease of filtering.
frequent_items['length'] = frequent_items['itemsets'].apply(lambda x: len(x))
frequent_items

# min_support=0.01--> 108 item set a frequent itemset is defined as a set of items that occur together in at least 1% of all transactions 
# min_support=0.02--> 37 item set  a frequent itemset is defined as a set of items that occur together in at least 2% of all transactions
# min_support=0.001--> 1575  item set  a frequent itemset is defined as a set of items that occur together in at least .1% of all transactions 

TypeError: ignored

In [ ]:
frequent_items[frequent_items['support'] == frequent_items.support.max()]

In [ ]:
# the most 10 frequently occurring item in our dataset with item set of given length  
def most_10_frequent_items(length):
    return frequent_items.sort_values('support', ascending=False)[frequent_items['length'] == length].head(10)

    

In [ ]:
# the most 10 frequently occurring item in our dataset with item set of length 1 

most_10_frequent_items(1)
# The output shows the Banana is the most frequently occurring item in our dataset

In [ ]:
# the most 10 frequently occurring item in our dataset with item set of length 2

most_10_frequent_items(2)
# The output shows that the Organic Strawberries & Bag of Organic Bananas combination are the most frequently occurring items when the length of the itemset is two.

In [ ]:
# the most 10 frequently occurring item in our dataset with item set of length 3 

most_10_frequent_items(3)

# The output shows the (Organic Hass Avocado, Organic Strawberries, Bag of Organic Bananas combination are the most frequently occurring items when the length of the itemset is three.

In [ ]:
# the most 10 frequently occurring item in our dataset with item set of length 4 

most_10_frequent_items(4)

# The output shows the 'Organic Hass Avocado', 'Organic Raspberries', 'Organic Strawberries', 'Bag of Organic Bananas' combination are the most frequently occurring items when the length of the itemset is four.

In [ ]:
#all items with a given length and at least support
def all_items_with_at_least_support_and_len(support , len):
    return frequent_items[ (frequent_items['length'] == len) & (frequent_items['support'] >= support) ]
    

In [ ]:
#all items with a length of 2, and the minimum support is more than 0.02 (occur together in at least 2% of all transactions)

all_items_with_at_least_support_and_len(0.02 , 2)

In [ ]:
#all items with a length of 1, and the minimum support is more than 0.1 (occur in at least 10% of all transactions)

all_items_with_at_least_support_and_len(0.1 , 1)

In [ ]:
# showing support of itemset of product
def show_itemset_support(frozenset):
    return frequent_items[ frequent_items['itemsets'] == frozenset ]
    
    

In [ ]:

# show_itemset_support({'Organic Strawberries', 'Organic Baby Spinach'})
# show_itemset_support({'Organic Hass Avocado','Bag of Organic Bananas'})
# show_itemset_support({'Organic Hass Avocado','Organic Strawberries','Bag of Organic Bananas'})
show_itemset_support({'Organic Hass Avocado','Organic Raspberries','Organic Strawberries','Bag of Organic Bananas'})


### The association rules :
are simply the if-else statements. The IF component of an association rule is known as the antecedent. The THEN component is known as the consequent. The antecedent and the consequent are disjoint; they have no items in common.

> 1) **Support**: This measure gives an idea of how frequent an itemset is in all the transactions,  the fraction of the total number of transactions in which the itemset occurs.<br>
2) **Confidence**: This says how likely item Y is purchased when item X is purchased, expressed as {X -> Y} , is the conditional probability of occurrence of consequent given the antecedent.<br>
3) **Lift**: This says how likely item Y is purchased when item X is purchased, while controlling for how popular item Y is.<br>
4) **Leverage** : how different is the co-occurrence of the antecedent X and the consequent Y of a rule from independence it takes values in the range [−0.25,0.25].<br>


> low confidence **-->** due to few purchases of male cosmetics in general<br>
left > 1  **-->**  item Y is likely to be bought if item X is bought<br>
left < 1  **-->**  item Y is unlikely to be bought if item X is bought.<br>
left = 1  **-->**  which implies no association between items<br>
Finally, it should be noted that Support values equal to unity implies Lift values close to unity, but the opposite is not necessarily true<br>

In [ ]:
# rules derived from the frequent itemsets only if the level of lift is above the 50 percent threshold
rules = association_rules(frequent_items, metric='lift', min_threshold=1.1)
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

# left = 1 --> 3950 rule
# left = 1.1 --> 3868 rule
# left = .5 --> 4126 rule


In [ ]:
# rules that satisfy the following criteria: at least 2 antecedents , a confidence > 0.75 & a lift score > 1.2
rules[ (rules['antecedent_len'] >= 1) &
       (rules['confidence'] >= .3) &
       (rules['lift'] >= 3) ]

rules based on huge num of items --> support
rules based on 
rules based on

In [ ]:
# sort rules ascending based on metric
def sorted_rules_asc_based_on_metric(metric='support'):
    return rules.sort_values(metric)

In [ ]:
sorted_rules_asc_based_on_metric('lift')

In [ ]:
rules['leverage'].min()
# rules['confidence'].min()
# rules['lift'].min()
# rules['leverage'].min()
# rules['conviction'].min()
# rules['antecedent_len'].max()

In [ ]:
# the fun. initialized with min. value for each metric
def rules_with_specific_threshold(support=.001,confidence=.005,lift=1.1,leverage=9.5e-05,conviction=1):
    return rules[ (rules['support'] >= support) &
       (rules['confidence'] >= confidence) &
       (rules['lift'] >= lift)&
       (rules['leverage'] >= leverage) &
       (rules['conviction'] >= conviction)  ]

In [ ]:
# rules_with_specific_threshold(lift=3)
rules_with_specific_threshold(support=.021,lift=1)



In [ ]:
def select_rules_with_antecedents_length(len):
    return rules[ rules['antecedent_len'] == len]

In [ ]:
select_rules_with_antecedents_length(3)

In [ ]:
# take set of antecedents names max names is 3
def select_rules_with_antecedents_names(names=set()):
    return rules[rules['antecedents'].apply(lambda x:  names in {x})]


In [ ]:
# select_rules_with_antecedents_names({'Small Hass Avocado'})
# select_rules_with_antecedents_names({'Broccoli Crown'})
# select_rules_with_antecedents_names({'Bag of Organic Bananas','Organic Hass Avocado'})
select_rules_with_antecedents_names({'Organic Hass Avocado'})

In [ ]:
# take set of consequents names max names is 3
def select_rules_with_consequents_names(names=set()):
    return rules[rules['consequents'].apply(lambda x:  names in {x})]


In [ ]:
# select_rules_with_consequents_names({'Small Hass Avocado'})
# select_rules_with_consequents_names({'Broccoli Crown'})
# select_rules_with_consequents_names({'Bag of Organic Bananas','Organic Hass Avocado'})
select_rules_with_consequents_names({'Organic Hass Avocado'})